In [9]:
import os
import numpy as np
import pandas as pd

import rocks
rocks.set_log_level("error")
import time as t

import requests


In [5]:
# DAMIT
path_damit = '../data/damit'
url = 'https://astro.troja.mff.cuni.cz/projects/damit/generated_files/open/AsteroidModel/'

# Read data

In [10]:
# Read DAMIT Tables
ssos = pd.read_csv(os.path.join( path_damit, 'damit-asteroids.csv'))
models = pd.read_csv(os.path.join( path_damit, 'damit-asteroid_models.csv'))
refs = pd.read_csv(os.path.join( path_damit, 'damit-asteroid_models_references.csv'))
bibs = pd.read_csv(os.path.join( path_damit, 'damit-references.csv'))

In [11]:
# Select columns for ASTEROIDS
ssos = ssos[ ['id', 'number', 'name', 'designation'] ]
ssos.columns = ['asteroid_id', 'number', 'name', 'designation']

In [12]:
# Select columns for MODELS
models = models[ [ 'id', 'asteroid_id', 'lambda', 'beta', 'period' ] ]
models.columns = [ 'model_id', 'asteroid_id', 'lambda', 'beta', 'period' ]

In [13]:
# Select columns for REFERENCES
refs = refs[ ['asteroid_model_id','reference_id'] ]
refs.columns = ['model_id','reference_id']

In [14]:
# Select columns for BIBLIOGRAPHY
bibs = bibs[ ['id', 'bibcode'] ]
bibs.columns = ['reference_id', 'bibcode']

In [15]:
# Merge everything
data = models.merge( ssos, on='asteroid_id')
data = data.merge( refs, on='model_id')
data = data.merge( bibs, on='reference_id')
data

,model_id,asteroid_id,lambda,beta,period,number,name,designation,reference_id,bibcode
0,101,101,35.0,-12.0,7.813230,2.0,Pallas,NaN,106,2003icar..164..346t
1,106,104,340.0,42.0,7.274471,6.0,Hebe,NaN,106,2003icar..164..346t
2,110,106,335.0,-5.0,12.866670,8.0,Flora,NaN,106,2003icar..164..346t
3,111,107,180.0,22.0,5.079176,9.0,Metis,NaN,106,2003icar..164..346t
4,117,111,98.0,57.0,7.443224,19.0,Fortuna,NaN,106,2003icar..164..346t
...,...,...,...,...,...,...,...,...,...,...
16312,16275,10853,326.0,-83.0,8.016800,99949.0,Miepgies,NaN,665,2023arxiv230510798d
16313,16276,10854,34.0,-42.0,143.500000,99973.0,NaN,NaN,665,2023arxiv230510798d
16314,16277,10854,194.0,-64.0,143.490000,99973.0,NaN,NaN,665,2023arxiv230510798d
16315,16278,10855,192.0,-52.0,7.071400,99990.0,NaN,NaN,665,2023arxiv230510798d


In [16]:
data[ data.number==135 ]

,model_id,asteroid_id,lambda,beta,period,number,name,designation,reference_id,bibcode
13,162,142,272.0,52.0,8.4006,135.0,Hertha,NaN,106,2003icar..164..346t
93,1799,142,276.0,53.0,8.4006,135.0,Hertha,NaN,169,2017a&a...601a.114h
277,162,142,272.0,52.0,8.4006,135.0,Hertha,NaN,127,2009mpbu...36...98t


# Download spins and shapes

In [24]:
for i, r in data.iterrows():
 
    url_shape = f"{url}{r['model_id']}/shape.obj"
    print(i, r['number'], r['name'], r['model_id'], url_shape)

    response = requests.get(url_shape)

    with open( os.path.join(path_damit, f"{r['model_id']:d}.obj"), mode="wb") as file:
        file.write(response.content)

    # if i==2:
    #     break


0 2.0 Pallas 101 https://astro.troja.mff.cuni.cz/projects/damit/generated_files/open/AsteroidModel/101/shape.obj
1 6.0 Hebe 106 https://astro.troja.mff.cuni.cz/projects/damit/generated_files/open/AsteroidModel/106/shape.obj
2 8.0 Flora 110 https://astro.troja.mff.cuni.cz/projects/damit/generated_files/open/AsteroidModel/110/shape.obj
3 9.0 Metis 111 https://astro.troja.mff.cuni.cz/projects/damit/generated_files/open/AsteroidModel/111/shape.obj
4 19.0 Fortuna 117 https://astro.troja.mff.cuni.cz/projects/damit/generated_files/open/AsteroidModel/117/shape.obj
5 21.0 Lutetia 120 https://astro.troja.mff.cuni.cz/projects/damit/generated_files/open/AsteroidModel/120/shape.obj
6 23.0 Thalia 122 https://astro.troja.mff.cuni.cz/projects/damit/generated_files/open/AsteroidModel/122/shape.obj
7 23.0 Thalia 123 https://astro.troja.mff.cuni.cz/projects/damit/generated_files/open/AsteroidModel/123/shape.obj
8 55.0 Pandora 139 https://astro.troja.mff.cuni.cz/projects/damit/generated_files/open/Asteroi

In [22]:
response.text

'v   -14.841491    42.873639   260.043496\nv   -19.032020    38.288645   260.908015\nv   -20.158936    33.794014   261.461482\nv   -20.158936    -0.655562   264.854829\nv    16.660465    -0.655562   261.228192\nv    26.276179     1.755535   260.043496\nv   -69.104199   -15.151249   261.461482\nv   -69.622910   -19.064176   261.795994\nv   -38.567551   -19.064176   264.854829\nv   -79.154345   -27.775403   259.999184\nv   -48.143732   -58.786016   259.999184\nv   -38.567551   -48.308824   261.974111\nv   -29.835036   -47.151063   261.228192\nv    28.219827     1.755535   259.724189\nv    28.554339     1.595664   259.642950\nv    80.683553     1.595664   243.829631\nv    52.259547    70.217647   243.829631\nv    52.114881    70.276296   243.866124\nv    48.453924    68.797055   245.162468\nv    46.336069    66.679635   246.070865\nv   -11.118410    50.647362   258.153283\nv    -6.792338    91.680752   245.162468\nv    -9.671317    88.332151   246.540051\nv   -11.118410    84.924468   247